In [1]:
# Create a new set of 2,000 random latitudes and longitudes.
import numpy as np

lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
coordinates = list(zip(lats, lngs))

In [2]:
# Get the nearest city using the citipy module.
from citipy import citipy
cities = [citipy.nearest_city(
    coordinate[0],
    coordinate[1]
).city_name for coordinate in coordinates]

In [3]:
# Perform an API call with the OpenWeatherMap.
import requests
from config import weather_api_key
import pprint

base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
city_urls = [f"{base_url}&q={city.replace(' ','+')}" for city in cities]
city_weather_responses = [requests.get(city_url).json() for city_url in city_urls]

In [14]:
# Retrieve the information from the API call:
information = {
    "City": [cwr['name'] for cwr in city_weather_responses if cwr['cod'] == 200],
    "Country": [cwr['sys']['country'] for cwr in city_weather_responses if cwr['cod'] == 200],
    "Lat": [cwr["coord"]["lat"] for cwr in city_weather_responses if cwr['cod'] == 200],
    "Lng": [cwr["coord"]["lon"] for cwr in city_weather_responses if cwr['cod'] == 200],
    "Max Temp": [cwr["main"]["temp_max"] for cwr in city_weather_responses if cwr['cod'] == 200],
    "Humidity": [cwr["main"]["humidity"] for cwr in city_weather_responses if cwr['cod'] == 200],
    "Cloudiness": [cwr["clouds"]["all"] for cwr in city_weather_responses if cwr['cod'] == 200],
    "Wind Speed": [cwr["wind"]["speed"] for cwr in city_weather_responses if cwr['cod'] == 200],
    "Current Description": [cwr['weather'][0]['description'] for cwr in city_weather_responses if cwr['cod'] == 200]
}

In [15]:
# Add the data to a new DataFrame.
import pandas as pd
df = pd.DataFrame(information)
df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Busselton,AU,-33.6500,115.3333,49.69,87,2,12.06,clear sky
1,Rikitea,PF,-23.1203,-134.9692,73.20,70,70,20.15,broken clouds
2,Busselton,AU,-33.6500,115.3333,49.69,87,2,12.06,clear sky
3,Port Keats,AU,-14.2500,129.5500,74.79,44,0,10.92,clear sky
4,Bambous Virieux,MU,-20.3428,57.7575,68.22,78,94,16.71,light rain


In [16]:
# Export the DataFrame as a CSV file
import os
df.to_csv(os.path.join("Weather_Database", "WeatherPy_Database.csv"))